<a href="https://www.kaggle.com/code/fluloeo/sum-art?scriptVersionId=291062016" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [8]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import ast
import matplotlib.pyplot as plt

/kaggle/input/articles/df_dict_test.csv


In [9]:
!pip install -U bitsandbytes accelerate transformers -q
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-4B-Instruct-2507",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

torch.cuda.empty_cache()
!pip install langchain==0.0.208  -q
from langchain.text_splitter import RecursiveCharacterTextSplitter
!pip install rouge-score evaluate -q
from rouge_score import rouge_scorer
import evaluate
rouge = evaluate.load('rouge')

!pip install bert-score -q
from bert_score import BERTScorer

scorer = BERTScorer(lang="en", model_type="bert-base-multilingual-cased")

!pip install longdocfactscore -q
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from longdocfactscore.ldfacts import LongDocFACTScore
ldfacts_scorer = LongDocFACTScore(device=device)
from tqdm.notebook import tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 123.2 MB/s eta 0:00:0000:010:01
cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [12]:
def append_summary_to_file(final_summary, filename='all_summaries.txt', directory='/kaggle/working/'):
    """
    Добавляет текст final_summary в конец указанного файла.

    Параметры:
        final_summary (str): Текст сводки для добавления в файл.
        filename (str): Имя файла (по умолчанию 'all_summaries.txt').
        directory (str): Путь к папке для сохранения (по умолчанию рабочая директория Kaggle).
    """
    # Формируем полный путь к файлу
    filepath = directory + filename

    # Открываем файл в режиме добавления ('a') с кодировкой UTF-8
    # Режим 'a' создаст файл, если его нет, и допишет текст в конец, если он существует
    with open(filepath, 'a', encoding='utf-8') as file:
        # Записываем текст и добавляем перенос строки для разделения записей
        file.write(final_summary + '\n')

    print(f'Текст добавлен в файл: {filepath}')


    

In [13]:
df = pd.read_csv('/kaggle/input/articles/df_dict_test.csv')
df

,id,title,abstract,dict_test
0,1602.04402,balanced truncation of linear time-invariant s...,this paper discusses model order reduction of ...,{'I. INTRODUCTION AND PROBLEM FORMULATIONS': '...
1,1611.01462,tying word vectors and word classifiers: a los...,recurrent neural networks have been very succe...,"{'INTRODUCTION': ""Neural network models have r..."
2,1611.04496,multi-view recurrent neural acoustic word embe...,recent work has begun exploring neural acousti...,{'INTRODUCTION': 'Word embeddings-continuous-v...
3,1808.00560,compressible spectral mixture kernels with spa...,spectral mixture (sm) kernels comprise a power...,"{'Introduction': ""Gaussian processes (GPs) con..."
4,2111.00405,limitations of the macaulay matrix approach fo...,recently chen and gao~\cite{chengao2017} propo...,{'Introduction': 'Solving systems of multivari...
...,...,...,...,...
95,2307.14341,virtual mirrors: non-line-of-sight imaging bey...,non-line-of-sight (nlos) imaging methods are c...,{'Computed image of': 'T-shaped object from a ...
96,2307.14354,learned gridification for efficient point clou...,neural operations that rely on neighborhood in...,{'Introduction': 'Point clouds provide sparse ...
97,2307.14362,learnable wavelet neural networks for cosmolog...,convolutional neural networks (cnns) have been...,{'Introduction': 'The process of extracting in...
98,2307.14392,human-centric scene understanding for 3d large...,human-centric scene understanding is significa...,{'Introduction': 'Human-centric scene understa...


*Пример для одной статьи:*

статьи 12 8 7 имеют проблемы с большим количеством чанков


In [14]:
from typing import List

def get_token_length(text: str) -> int:
    return len(tokenizer.encode(text))

def merge_small_chunks_by_tokens(chunks: List[str], min_tokens: int) -> List[str]:
    """
    Объединяет чанки, если их длина в ТОКЕНАХ меньше min_tokens.
    Слияние происходит с наименьшим соседом (по количеству токенов).
    """
    processed_chunks = chunks[:]
    separator = " " # Разделитель при склейке
    
    i = 0
    while i < len(processed_chunks):
        current_chunk = processed_chunks[i]
        current_len = get_token_length(current_chunk)
        if current_len >= min_tokens:
            i += 1
            continue
        if len(processed_chunks) == 1:
            break
        if i > 0:
            left_len = get_token_length(processed_chunks[i-1])
        else:
            left_len = float('inf')

        if i < len(processed_chunks) - 1:
            right_len = get_token_length(processed_chunks[i+1])
        else:
            right_len = float('inf')
        if left_len < right_len:
            # print(f"DEBUG: Слияние '{processed_chunks[i][:20]}...' ВЛЕВО (Tokens: {current_len} + {left_len})")
            
            processed_chunks[i-1] = processed_chunks[i-1] + separator + processed_chunks[i]
            processed_chunks.pop(i)
            i -= 1 
        else:
            # print(f"DEBUG: Слияние '{processed_chunks[i][:20]}...' ВПРАВО (Tokens: {current_len} + {right_len})")
            processed_chunks[i] = processed_chunks[i] + separator + processed_chunks[i+1]
            processed_chunks.pop(i+1)
    return processed_chunks




In [15]:
def merge_chunks_in_dataset(df=df, min_tokens=100, n=100, n_threshold = 10):
    data = pd.DataFrame(columns=['chunks'])
    max_len_chunks = []
    max_size_chunks = []
    min_size_chunks = []
    for i in range(n):
        data_dict = ast.literal_eval(df['dict_test'].iloc[i])
        tokens_chunks_list_0 = list(map(lambda x: get_token_length(x), list(data_dict.values())))
        if len(tokens_chunks_list_0)>n_threshold or min(tokens_chunks_list_0)<min_tokens:
            final_chunks = merge_small_chunks_by_tokens(list(data_dict.values()), min_tokens=min_tokens)
            # print(f"\nКонечное количество чанков: {len(final_chunks)}\n")
            tokens_chunks_list = list(map(lambda x: get_token_length(x), final_chunks))
            # print('Максимальный размер чанка',max(tokens_chunks_list),'\n')
            max_len_chunks.append(len(final_chunks))
            max_size_chunks.append(max(tokens_chunks_list))
            min_size_chunks.append(min(tokens_chunks_list))
            data.loc[len(data)] = [final_chunks]
        else:
            print("Ничего не меняем, кол-во чанков, мин токенов", len(tokens_chunks_list_0),min(tokens_chunks_list_0))
            data.loc[len(data)] = [data_dict]
    print('Максимальное Кол-во чанков из всех статей\n',(max(max_len_chunks)))
    print('Минимальное Кол-во чанков из всех статей\n',(min(max_len_chunks)))
    print('Максимальный размер чанка из всех статей\n',max(max_size_chunks))
    print('Минимальный размер чанка из всех статей\n',min(min_size_chunks))
    return data
data = merge_chunks_in_dataset(df=df, min_tokens=700, n=20,n_threshold = 10)

Максимальное Кол-во чанков из всех статей
 14
Минимальное Кол-во чанков из всех статей
 5
Максимальный размер чанка из всех статей
 3331
Минимальный размер чанка из всех статей
 706


Функция для того чтобы отправить инструкцию модели

In [16]:
def qwen(article, prompt, max_tokens):
    torch.cuda.empty_cache()
    final_prompt = f"{prompt}:\n\n{article}"
    messages = [{"role": "user", "content": final_prompt}]
    model.eval()
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
            truncation=False
        ).to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False
        )
    final_summary = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return final_summary

Функция для обработки чанков статьи, встроенный text_splitter можно отключить

In [17]:
def summarize(article, prompt_0, prompt_1,chunk_size=1500, max_tokens_0=700, max_tokens_1=700, text_splitter = True):
    torch.cuda.empty_cache()
    if text_splitter:
        text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer=tokenizer,
            chunk_size=chunk_size,
            chunk_overlap=chunk_size//20,
            separators=["\n\n", "\n", ". "],
            keep_separator=True
        )
        article_chunks = text_splitter.split_text(article)
    else:
        article_chunks = article
    summaries = []
    for i, chunk in tqdm(enumerate(article_chunks), total=len(article_chunks)):
        try:
            chunk_summary = qwen(article=chunk, prompt=prompt_0, max_tokens=max_tokens_0)
            # print(f"\nSummary chunk number {i}")
            # print(chunk_summary)
        except torch.cuda.OutOfMemoryError as e:
            print(f"⚠️ Ошибка нехватки памяти CUDA, слишком длинный чанк: {str(e)[:100]}...")
            return 'ERROR'

        summaries.append(chunk_summary)
        torch.cuda.empty_cache()
    combined_summary = "\n".join(summaries)
    
    try:
        final_summary = qwen(article=combined_summary, prompt=prompt_1, max_tokens=max_tokens_1)
    except torch.cuda.OutOfMemoryError as e:
        print(f"⚠️ Ошибка нехватки памяти CUDA при итоговой суммаризации: {str(e)[:100]}...")
        return 'ERROR'
    return final_summary

In [18]:
rouge_list = []
bert_list = []
ldfacts_list = []
summaries_list = []

def to_full_text(data_dict):
    text = ''
    for key, value in data_dict.items(): 
        text += f"{key}\n\n{value}\n\n"
    return text


for i in range(20):
    print(f"\n=== Статья {i} ===")
    #извлекаем чанки, абстракт, названия разделов
    article_dict = ast.literal_eval(df['dict_test'].iloc[i]) #словарь для i-той статьи 
    chunked_article = data['chunks'].iloc[i] #list(article_dict.values()) #список чанков !!!!ВАЖНО я поменял на новый list 
    # chapters_list = list(article_dict.keys()) #список глав
    # print('Список разделов')
    # display(pd.DataFrame(chapters_list))
    abstract = df['abstract'].iloc[i] 
    article = to_full_text(article_dict) #полный текст статьи с ключами и значениями словаря
    
    print(f"Токенов в статье: {len(tokenizer.encode('\n'.join(chunked_article)))}")
    prompt_0 = "You're a science editor. Briefly summarize this fragment of the scientific text in original language. Do not add information that is not in the source texts.Try to answer in 350 words. Your answer should be perfect. Do not add information that is not in the source texts"
    prompt_1 = "You're a science editor. Based on the following summaries of the parts of the article, create a single, coherent and concise summary of the entire scientific article in original language, highlighting the common goal, methods, key results and conclusion. Try to answer with a length of 600 to 650 words. Your answer should be perfect. Do not add information that is not in the source texts"
    
    summary_full = summarize(chunked_article, prompt_0 = prompt_0, prompt_1 = prompt_1, max_tokens_0 = 400, max_tokens_1 = 700, text_splitter = False)
    print(f'\nSummary article number {i}')
    print(f"Токенов в summary: {len(tokenizer.encode(summary_full))}")
    print(summary_full)
    summaries_list.append({
        'article_id': i,
        'original_text': article,
        'abstract': abstract,
        'summary': summary_full
    })
    append_summary_to_file(f"--- Summury #{i} ---")
    append_summary_to_file(summary_full)
    append_summary_to_file("")
    
    results = rouge.compute(
    predictions=[summary_full],
    references=[article],
    use_stemmer=True
    )
    print("ROUGE Metrics:")

    rouge_dict = {}
    for key, value in results.items():
        print(f"{key}: {value:.4f}")
        rouge_dict[key] = value
    rouge_list.append(rouge_dict)

    _, _, F1 = scorer.score([summary_full], [abstract])
    bert_f1 = F1.item()
    print(f"\nBERTScore: F1 = {bert_f1:.4f}")
    bert_list.append(bert_f1)

    ldfacts_sum = ldfacts_scorer.score_src_hyp_long([article], [summary_full])
    ldfacts_abs = ldfacts_scorer.score_src_hyp_long([article], [abstract])
    print(f"LongDocFACTScore for sum: {ldfacts_sum[0]}")
    print(f"LongDocFACTScore for abstract: {ldfacts_abs[0]}")
    ldfacts_list.append({
        'for_summary': ldfacts_sum[0],
        'for_abstract': ldfacts_abs[0]
    })


=== Статья 0 ===
Токенов в статье: 6901


  0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Summary article number 0
Токенов в summary: 511
This article addresses the challenge of model order reduction (MOR) for linear time-invariant continuous-time systems with large order, where computational and storage demands hinder practical simulation and design. While standard balanced truncation (FIBT) preserves stability and offers a priori error bounds over the entire frequency range, it is suboptimal when input signals operate within a finite frequency interval [ω₁, ω₂]. To improve in-band approximation performance, several balancing-related methods are analyzed: singular perturbation approximation (SPA), frequency-weighted balanced truncation (FWBT), and frequency-limited Grammians balanced truncation (FGBT). However, these methods either lack error bounds for finite intervals, require iterative design, or fail due to non-positive semi-definite Grammians. A key observation is that existing methods evaluate performance using entire-frequency indices, despite the finite-frequency 

  0%|          | 0/5 [00:00<?, ?it/s]

...


Summary article number 17
Токенов в summary: 537
This study identifies a novel class of attacks that exploit imperceptible text encoding variations to manipulate search engine results and downstream AI systems, posing a significant threat to information integrity. The core goal is to demonstrate how adversaries can deceive search engines—both commercial (e.g., Google, Bing) and open-source (e.g., Elasticsearch)—by introducing subtle, visually undetectable perturbations in text encoding, without altering the visible content. These perturbations—such as invisible characters (e.g., Zero Width Space), homoglyphs (e.g., Latin H vs. Cyrillic Н), reordering (e.g., Right-to-Left Override), and deletions—produce identical rendered text but differ in binary encoding. By exploiting the lack of cross-representation awareness in text processing pipelines, adversaries can either hide malicious content from standard searches (hiding potential) or push false content into results (surfacing potential)

  0%|          | 0/9 [00:00<?, ?it/s]


Summary article number 18
Токенов в summary: 522
This article presents VoxTracer, a novel traceable voice conversion (VC) framework designed to enable robust speaker identity recovery from degraded, compressed, or processed speech—addressing critical gaps in current VC systems where source traceability is absent. The core goal is to enable reliable source speaker identification and original speech restoration in voice conversion, countering misuse such as deepfake fraud, reputational harm, and financial manipulation. Unlike traditional audio watermarking, which embeds fragile, error-prone signals and fails under compression or processing, VoxTracer integrates speaker identity imperceptibly into the VC process using speaker embeddings and a VAE-Glow architecture. It operates in four stages: voice conversion, identity hiding, transmission, and tracing. During hiding, a speaker embedding is encoded into a Gaussian-distributed latent code via a VAE encoder and embedded into the speech gen

  0%|          | 0/9 [00:00<?, ?it/s]


Summary article number 19
Токенов в summary: 621
The article presents TriMLP, a novel sequential recommendation model that addresses the fundamental incompatibility between fully-connected MLP architectures and auto-regressive training paradigms. While recent all-MLP models like MLP-Mixer and ResMLP have gained popularity for their computational efficiency, empirical evaluations on 12 public datasets—including Beauty, Sports, MovieLens, and LBSN benchmarks—reveal that standard MLPs underperform significantly, lagging NextItNet by up to 28.19% in HR@10. This degradation is attributed to the fully-connected design enabling antichronological interactions (future-to-past), which violates the autoregressive training principle where tokens should only attend to themselves and previous interactions. The authors demonstrate that MLPs are insensitive to sequence order, confirming their inability to model chronological user preferences effectively.

To resolve this, TriMLP introduces a triangul

In [19]:
print("\n--- ROUGE Metrics (средние значения) ---")
for key in rouge_list[0].keys():
    avg = sum(r[key] for r in rouge_list) / len(rouge_list)
    print(f"{key}: {avg:.4f}")

avg_bert = sum(bert_list) / len(bert_list)
print(f"\n--- BERTScore F1 (среднее): {avg_bert:.4f} ---")

avg_sum = sum(l['for_summary'] for l in ldfacts_list) / len(ldfacts_list)
avg_abs = sum(l['for_abstract'] for l in ldfacts_list) / len(ldfacts_list)

print(f"\n--- LongDocFACTScore (средние) ---")
print(f"Для суммаризаций: {avg_sum:.4f}")
print(f"Для абстрактов: {avg_abs:.4f}")


--- ROUGE Metrics (средние значения) ---
rouge1: 0.1151
rouge2: 0.0510
rougeL: 0.0641
rougeLsum: 0.0965

--- BERTScore F1 (среднее): 0.6845 ---

--- LongDocFACTScore (средние) ---
Для суммаризаций: -4.6309
Для абстрактов: -4.7052


In [32]:
pd.DataFrame(summaries_list).to_csv('summaries.csv')

data_load = pd.read_csv('/kaggle/working/summaries.csv')
data_load.head()

,Unnamed: 0,article_id,original_text,abstract,summary
0,0,0,I. INTRODUCTION AND PROBLEM FORMULATIONS\n\nWe...,this paper discusses model order reduction of ...,This article addresses the challenge of model ...
1,1,1,INTRODUCTION\n\nNeural network models have rec...,recurrent neural networks have been very succe...,This article proposes a novel loss framework a...
2,2,2,INTRODUCTION\n\nWord embeddings-continuous-val...,recent work has begun exploring neural acousti...,This paper proposes a multi-view learning fram...
3,3,3,Introduction\n\nGaussian processes (GPs) const...,spectral mixture (sm) kernels comprise a power...,This paper proposes a novel structured station...
4,4,4,Introduction\n\nSolving systems of multivariat...,recently chen and gao~\cite{chengao2017} propo...,The article addresses the problem of solving s...
